# Sudoku Puzzle Generator Notebook

Generates valid Sudoku puzzles of sizes 4×4 (2×2 boxes) and 9×9 (3×3 boxes) using Z3 SAT solver.

## Installations

In [ ]:
!pip install z3-solver

In [ ]:
from z3 import *
import numpy as np
import random
import time
import json
import os

## Sudoku Solver with Z3

Unlike KenKen, Sudoku has:
- No cage operations (just given cells)
- Box constraints (2×2 for 4×4 grids, 3×3 for 9×9 grids)

In [ ]:
def solve_sudoku(size, given_cells):
    """
    Solve a Sudoku puzzle using Z3.
    
    Args:
        size: Grid size (4 or 9)
        given_cells: dict of {(row, col): value} for pre-filled cells
    
    Returns:
        Solution grid or None if unsolvable
    """
    box_size = 2 if size == 4 else 3
    
    # Create variables
    X = [[Int(f"x_{i}_{j}") for j in range(size)] for i in range(size)]
    
    s = Solver()
    
    # Cell range constraints: 1 <= X[i][j] <= size
    for i in range(size):
        for j in range(size):
            s.add(And(X[i][j] >= 1, X[i][j] <= size))
    
    # Given cell constraints
    for (i, j), val in given_cells.items():
        s.add(X[i][j] == val)
    
    # Row constraints: each row has distinct values
    for i in range(size):
        s.add(Distinct([X[i][j] for j in range(size)]))
    
    # Column constraints: each column has distinct values
    for j in range(size):
        s.add(Distinct([X[i][j] for i in range(size)]))
    
    # Box constraints: each box has distinct values
    for box_row in range(box_size):
        for box_col in range(box_size):
            cells = []
            for i in range(box_size):
                for j in range(box_size):
                    cells.append(X[box_row * box_size + i][box_col * box_size + j])
            s.add(Distinct(cells))
    
    if s.check() == sat:
        m = s.model()
        solution = [[m.evaluate(X[i][j]).as_long() for j in range(size)] for i in range(size)]
        return solution
    else:
        return None

In [ ]:
def has_unique_solution(size, given_cells):
    """
    Check if a Sudoku puzzle has exactly one solution.
    """
    box_size = 2 if size == 4 else 3
    
    X = [[Int(f"x_{i}_{j}") for j in range(size)] for i in range(size)]
    s = Solver()
    
    # Cell range constraints
    for i in range(size):
        for j in range(size):
            s.add(And(X[i][j] >= 1, X[i][j] <= size))
    
    # Given cell constraints
    for (i, j), val in given_cells.items():
        s.add(X[i][j] == val)
    
    # Row constraints
    for i in range(size):
        s.add(Distinct([X[i][j] for j in range(size)]))
    
    # Column constraints
    for j in range(size):
        s.add(Distinct([X[i][j] for i in range(size)]))
    
    # Box constraints
    for box_row in range(box_size):
        for box_col in range(box_size):
            cells = []
            for i in range(box_size):
                for j in range(box_size):
                    cells.append(X[box_row * box_size + i][box_col * box_size + j])
            s.add(Distinct(cells))
    
    # Find first solution
    if s.check() != sat:
        return False
    
    first_solution = s.model()
    
    # Block first solution and check for another
    block = []
    for i in range(size):
        for j in range(size):
            block.append(X[i][j] != first_solution.evaluate(X[i][j]))
    s.add(Or(block))
    
    # If no second solution exists, puzzle is unique
    return s.check() == unsat

## Generate Complete Sudoku Grid

In [ ]:
def generate_complete_grid(size):
    """
    Generate a complete, valid Sudoku grid using Z3 with randomization.
    """
    box_size = 2 if size == 4 else 3
    
    X = [[Int(f"x_{i}_{j}") for j in range(size)] for i in range(size)]
    s = Solver()
    
    # Cell range constraints
    for i in range(size):
        for j in range(size):
            s.add(And(X[i][j] >= 1, X[i][j] <= size))
    
    # Row constraints
    for i in range(size):
        s.add(Distinct([X[i][j] for j in range(size)]))
    
    # Column constraints
    for j in range(size):
        s.add(Distinct([X[i][j] for i in range(size)]))
    
    # Box constraints
    for box_row in range(box_size):
        for box_col in range(box_size):
            cells = []
            for i in range(box_size):
                for j in range(box_size):
                    cells.append(X[box_row * box_size + i][box_col * box_size + j])
            s.add(Distinct(cells))
    
    # Add some random constraints to get varied solutions
    random_cells = random.sample([(i, j) for i in range(size) for j in range(size)], min(3, size))
    for (i, j) in random_cells:
        val = random.randint(1, size)
        s.add(X[i][j] == val)
    
    if s.check() == sat:
        m = s.model()
        return [[m.evaluate(X[i][j]).as_long() for j in range(size)] for i in range(size)]
    else:
        # If random constraints conflict, try again without them
        return generate_complete_grid(size)

## Generate Puzzle by Removing Cells

In [ ]:
def generate_puzzle(size, min_clues=None, max_clues=None):
    """
    Generate a Sudoku puzzle by starting with a complete grid and removing cells
    while maintaining unique solution.
    
    Args:
        size: Grid size (4 or 9)
        min_clues: Minimum number of given cells (default: size+1 for 4x4, 17 for 9x9)
        max_clues: Maximum number of given cells
    
    Returns:
        (puzzle, solution) tuple where puzzle has 0s for empty cells
    """
    if min_clues is None:
        min_clues = 4 if size == 4 else 17  # 17 is theoretical minimum for 9x9
    if max_clues is None:
        max_clues = size * size // 2
    
    # Generate complete solution
    solution = generate_complete_grid(size)
    
    # Start with all cells as given
    puzzle = [row[:] for row in solution]
    given_cells = {(i, j): solution[i][j] for i in range(size) for j in range(size)}
    
    # Randomly order cells for removal
    cells_to_try = [(i, j) for i in range(size) for j in range(size)]
    random.shuffle(cells_to_try)
    
    for (i, j) in cells_to_try:
        if len(given_cells) <= min_clues:
            break
        
        # Try removing this cell
        val = given_cells.pop((i, j))
        
        # Check if puzzle still has unique solution
        if has_unique_solution(size, given_cells):
            puzzle[i][j] = 0  # Mark as empty
        else:
            # Put it back
            given_cells[(i, j)] = val
    
    return puzzle, solution

## Generate Puzzle Set

In [ ]:
def generate_puzzle_set(num_puzzles, size):
    """
    Generate a set of Sudoku puzzles.
    
    Args:
        num_puzzles: Number of puzzles to generate
        size: Grid size (4 or 9)
    
    Returns:
        (puzzles, solutions) tuple
    """
    puzzles = []
    solutions = []
    
    for i in range(num_puzzles):
        puzzle, solution = generate_puzzle(size)
        puzzles.append(puzzle)
        solutions.append(solution)
        print(f"Generated puzzle {i+1}/{num_puzzles}")
    
    return puzzles, solutions

## Demo: Generate Single Puzzle

In [ ]:
# Generate a single 4x4 puzzle
puzzle_4x4, solution_4x4 = generate_puzzle(4)

print("4x4 Puzzle:")
for row in puzzle_4x4:
    print(row)

print("\nSolution:")
for row in solution_4x4:
    print(row)

In [ ]:
# Generate a single 9x9 puzzle
puzzle_9x9, solution_9x9 = generate_puzzle(9)

print("9x9 Puzzle:")
for row in puzzle_9x9:
    print(row)

print("\nSolution:")
for row in solution_9x9:
    print(row)

## Generate Full Dataset

In [ ]:
puzzles_full = {}
solutions_full = {}
runtimes = {}

# Generate 4x4 puzzles
print("Generating 4x4 puzzles...")
start = time.time()
p, s = generate_puzzle_set(100, 4)
runtimes[4] = time.time() - start
puzzles_full[4] = p
solutions_full[4] = s

# Generate 9x9 puzzles
print("\nGenerating 9x9 puzzles...")
start = time.time()
p, s = generate_puzzle_set(100, 9)
runtimes[9] = time.time() - start
puzzles_full[9] = p
solutions_full[9] = s

print(f"\n4x4 runtime: {runtimes[4]:.2f}s")
print(f"9x9 runtime: {runtimes[9]:.2f}s")

## Plot Runtimes

In [ ]:
import matplotlib.pyplot as plt

sizes = sorted(runtimes.keys())
times = [runtimes[s] for s in sizes]

plt.figure(figsize=(8, 5))
plt.bar(sizes, times, color=['blue', 'green'])
plt.title("Runtime to Generate 100 Sudoku Puzzles by Grid Size")
plt.xlabel("Grid Size (n × n)")
plt.ylabel("Runtime (seconds)")
plt.xticks(sizes, ['4×4', '9×9'])
plt.grid(True, axis='y')
plt.tight_layout()
plt.show()

## Save Dataset

In [ ]:
# Prepare data for JSON serialization
output_data = {}
for size in puzzles_full:
    output_data[str(size)] = [
        {
            "puzzle": puzzles_full[size][i],
            "solution": solutions_full[size][i]
        }
        for i in range(len(puzzles_full[size]))
    ]

# Save to file
output_path = "./puzzles/puzzles_dict.json"
os.makedirs(os.path.dirname(output_path), exist_ok=True)

with open(output_path, "w") as f:
    json.dump(output_data, f, indent=2)

print(f"Saved {sum(len(v) for v in output_data.values())} puzzles to {output_path}")

## Verify Dataset

In [ ]:
# Load and verify
with open(output_path, "r") as f:
    loaded = json.load(f)

print("Dataset summary:")
for size, puzzles in loaded.items():
    print(f"  {size}×{size}: {len(puzzles)} puzzles")
    
    # Count average clues
    total_clues = 0
    for p in puzzles:
        clues = sum(1 for row in p['puzzle'] for cell in row if cell != 0)
        total_clues += clues
    avg_clues = total_clues / len(puzzles)
    print(f"         Average clues: {avg_clues:.1f}")